In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/zeya/Git/kth-machine_learning/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python",project="airqualitypredictionssr")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-25 20:22:23,750 INFO: Initializing external client
2025-11-25 20:22:23,750 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-25 20:22:25,170 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279183


'{"country": "australia", "city": "western", "street": "kalgoorlie", "aqicn_url": "https://api.waqi.info/feed/@10147", "latitude": "-30.759847", "longitude": "121.451105"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [34]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,6.0,australia,western,kalgoorlie,2025-11-25,https://api.waqi.info/feed/@10147


In [35]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


In [29]:
# import pandas as pd
# import numpy as np

# # 获取今天的数据
# aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
# print("原始 aq_today_df:")
# print(aq_today_df)

# # 直接修改 DataFrame，添加从 2025-11-18 到 2025-11-24 的数据
# dates = pd.date_range(start="2025-11-19", end="2025-11-24")
# pm25_values = [8.0, 9.0, 7.0, 2.0, 4.0, 8.0, 2.0]  # 手动设定或模拟的值

# # 用原来的行作为模板
# template_row = aq_today_df.iloc[0]

# # 清空原来的行，先生成空 DataFrame
# aq_today_df = pd.DataFrame(columns=aq_today_df.columns)

# # 添加每一行
# for date, pm25 in zip(dates, pm25_values):
#     new_row = template_row.copy()
#     new_row["date"] = date
#     new_row["pm25"] = float(pm25)  # 确保类型是 float
#     aq_today_df = pd.concat([aq_today_df, pd.DataFrame([new_row])], ignore_index=True)

# # 确认类型匹配
# aq_today_df['pm25'] = aq_today_df['pm25'].astype(np.float32)
# aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])
# aq_today_df['country'] = aq_today_df['country'].astype(str)
# aq_today_df['city'] = aq_today_df['city'].astype(str)
# aq_today_df['street'] = aq_today_df['street'].astype(str)
# aq_today_df['url'] = aq_today_df['url'].astype(str)

# print("修改后的 aq_today_df:")
# print(aq_today_df)

原始 aq_today_df:
   pm25    country     city      street       date  \
0   6.0  australia  western  kalgoorlie 2025-11-25   

                                 url  
0  https://api.waqi.info/feed/@10147  
修改后的 aq_today_df:
   pm25    country     city      street       date  \
0   8.0  australia  western  kalgoorlie 2025-11-19   
1   9.0  australia  western  kalgoorlie 2025-11-20   
2   7.0  australia  western  kalgoorlie 2025-11-21   
3   2.0  australia  western  kalgoorlie 2025-11-22   
4   4.0  australia  western  kalgoorlie 2025-11-23   
5   8.0  australia  western  kalgoorlie 2025-11-24   

                                 url  
0  https://api.waqi.info/feed/@10147  
1  https://api.waqi.info/feed/@10147  
2  https://api.waqi.info/feed/@10147  
3  https://api.waqi.info/feed/@10147  
4  https://api.waqi.info/feed/@10147  
5  https://api.waqi.info/feed/@10147  


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [30]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates -30.75°N 121.5°E
Elevation 363.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-25,19.750000,0.0,6.763786,205.201096,western
1,2025-11-26,23.200001,0.0,5.116561,140.710617,western
2,2025-11-27,22.700001,0.0,30.364031,169.755966,western
3,2025-11-28,21.750000,0.0,15.790833,133.152405,western
4,2025-11-29,22.450001,0.0,18.079027,144.727493,western
5,2025-11-30,24.650000,0.0,12.031756,128.927551,western
6,2025-12-01,28.299999,0.0,9.504273,114.623550,western


In [31]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [36]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-25 21:14:49,628 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279183/fs/1283141/fg/1774953


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279183/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 779274
         }
       },
       "result": {
         "observed_value": 6.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-25T08:14:49.000627Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [33]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-25 21:10:53,989 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279183/fs/1283141/fg/1774954


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279183/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-25 21:11:11,180 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-25 21:11:14,375 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-25 21:12:50,501 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-25 21:12:50,665 INFO: Waiting for log aggregation to finish.
2025-11-25 21:12:59,312 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 779275
         }
       },
       "result": {
         "observed_value": 5.116561412811279,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-25T08:10:53.000988Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
